In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
device = 'cuda:0'

In [17]:
data_path = r'D:\Datasets\fruits\5857_1166105_bundle_archive\fruits-360\Training'
data_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
train_data = datasets.ImageFolder(data_path, data_transform)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=500, pin_memory=True)

In [18]:
model = models.alexnet(pretrained=True)

In [19]:
model = model.to(device)

In [13]:
model.classifier[6].out_features = 131
for param in model.features.parameters():
    param.requires_grad = False

In [14]:
optimizer = optim.SGD(model.classifier.parameters(), lr=0.001, momentum=0.9)
# loss function
criterion = nn.CrossEntropyLoss()

In [20]:
from tqdm import tqdm_notebook
def fit(model, train_dataloader):
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    for i, data in enumerate(tqdm_notebook(train_dataloader)):
        data, target = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_running_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        train_running_correct += (preds == target).sum().item()
        loss.backward()
        optimizer.step()
    train_loss = train_running_loss/len(train_dataloader.dataset)
    train_accuracy = 100. * train_running_correct/len(train_dataloader.dataset)
 
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}')
    
    return train_loss, train_accuracy

In [21]:
import time
train_loss , train_accuracy = [], []
start = time.time()
for epoch in range(1):
    train_epoch_loss, train_epoch_accuracy = fit(model, trainloader)
    train_loss.append(train_epoch_loss)
    train_accuracy.append(train_epoch_accuracy)
end = time.time()
 
print((end-start)/60, 'minutes')

RuntimeError: CUDA out of memory. Tried to allocate 124.00 MiB (GPU 0; 4.00 GiB total capacity; 2.56 GiB already allocated; 2.61 MiB free; 2.91 GiB reserved in total by PyTorch)